In [12]:
from sklearn.neural_network import MLPClassifier
from  keras.wrappers.scikit_learn  import  KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from scipy import ndimage
from sklearn.ensemble import AdaBoostClassifier
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:


#!ls "/content/gdrive/MyDrive/Shopee"
path = ('/content/gdrive/MyDrive/colab_train.csv')
image_path = ('/content/gdrive/MyDrive/colab/')

In [15]:
df_trains  = pd.read_csv(path)
feature_cols = ['posting_id','image','image_phash','title']
y = df_trains['label_group']
X = df_trains[feature_cols]
#y_trains = df_trains['label_group']

In [16]:
train_files = []
y_train = []
image_width = 640
image_height = 640
i=0
image_path = image_path + df_trains['image']

In [17]:
import glob
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image 
from scipy import ndimage, misc
import cv2
from skimage.transform import resize
for i in range(len(df_trains)):
    y_label = df_trains['label_group']
    print(y_label.shape)
    new_X = []
    for i in range(len(df_trains)):
        userImage = cv2.imread(image_path[i])
        userImage = cv2.resize(userImage,(64,64))
        userImage = cv2.cvtColor(userImage,cv2.COLOR_BGR2GRAY)

        print(np.reshape(userImage,  4096))
        new_X.append(np.reshape(userImage,  4096))

Streaming output truncated to the last 5000 lines.
(99,)
[ 95 110 112 ... 163 126 155]
[255 255 255 ... 255 255 255]
[202 200 203 ... 243 246 245]
[254 254 254 ... 254 253 255]
[255 255 255 ...  65  65  65]
[177 177 147 ... 254 254 254]
[214 217 217 ... 232 229 229]
[250 251 250 ...  70 143 168]
[255 254 254 ... 245 245 242]
[29 29 29 ... 29 29 29]
[255 214 214 ... 232 212 255]
[255 255 250 ... 250 255 255]
[255 255 250 ... 250 255 255]
[255 255 255 ... 252 253 254]
[119 118 118 ...  61  61  61]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[225 226 225 ... 179 181 177]
[5 5 7 ... 6 5 5]
[111 110 109 ... 166 163 163]
[234 232 230 ... 234 234 235]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[254 254 254 ... 254 254 254]
[

In [18]:
normalized = preprocessing.scale(new_X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(normalized, y_label, test_size=0.3, random_state=2)

In [20]:
my_decisiontree = DecisionTreeClassifier(random_state=2)
my_decisiontree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2, splitter='best')

In [21]:
y_predict_dt = my_decisiontree.predict(X_test)
score_dt = accuracy_score(y_test, y_predict_dt)
print('Decision tree accuracy: ')
print(score_dt)

y_predict_dt_proba = my_decisiontree.predict_proba(X_test)

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_dt_proba[:,1], pos_label=1)

dt_AUC = metrics.auc(fpr, tpr)
print('Decision tree AUC: ')
print(dt_AUC)

Decision tree accuracy: 
0.06666666666666667
Decision tree AUC: 
nan


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [23]:
from sklearn.utils import resample
tree_predictions = []
for i in range(29):
    bootstarp_size = int(0.8*len(df_trains))
    x_bag, y_bag = resample(X_train, y_train, n_samples = bootstarp_size , random_state=i , replace = True) 
    Base_DecisionTree = DecisionTreeClassifier(random_state=2)
    Base_DecisionTree.fit(x_bag, y_bag)
    y_predict_bag = Base_DecisionTree.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict_bag)
    tree_predictions.append(accuracy)
print(tree_predictions)


def Average(tree_predictions): 
    return sum(tree_predictions) / len(tree_predictions)

bagging_avg = Average(tree_predictions)


[0.0, 0.1, 0.03333333333333333, 0.06666666666666667, 0.0, 0.03333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.03333333333333333, 0.0, 0.06666666666666667, 0.0, 0.06666666666666667, 0.03333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.03333333333333333, 0.06666666666666667, 0.03333333333333333, 0.03333333333333333, 0.0, 0.06666666666666667, 0.06666666666666667, 0.03333333333333333, 0.06666666666666667, 0.06666666666666667]


In [24]:
def vote(input):
    member = max(set(input), key=input.count)
    return member
vote_y_test = vote(tree_predictions)
print('Bagging voting: ')
print(vote_y_test)
print('Bagging accuracy mean: ')
print(bagging_avg)

Bagging voting: 
0.06666666666666667
Bagging accuracy mean: 
0.047126436781609195


In [1]:
from sklearn.model_selection import GridSearchCV

In [16]:
seed = 1  
np.random.seed(seed)

neuron_number = [(i,) for i in range(5,91, 5)]

param_grid = dict(hidden_layer_sizes = neuron_number)
print(param_grid,'\n')

my_ANN_grid = MLPClassifier(hidden_layer_sizes=(30,), activation= 'logistic', 
                       solver='adam', alpha=1, random_state=1, 
                       learning_rate_init = 0.02, verbose=True, tol=0.0001)

grid = GridSearchCV(my_ANN_grid, param_grid, cv=3, scoring='accuracy')

grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_params_)

{'hidden_layer_sizes': [(5,), (10,), (15,), (20,), (25,), (30,), (35,), (40,), (45,), (50,), (55,), (60,), (65,), (70,), (75,), (80,), (85,), (90,)]} 



/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Iteration 1, loss = 3.88552233
Iteration 2, loss = 37.98154665
Iteration 3, loss = 61.01552549
Iteration 4, loss = 81.24359057
Iteration 5, loss = 90.53068322
Iteration 6, loss = 86.61838239
Iteration 7, loss = 75.99980411
Iteration 8, loss = 64.39903490
Iteration 9, loss = 54.00130341
Iteration 10, loss = 44.22751239
Iteration 11, loss = 34.72628751
Iteration 12, loss = 26.48378340
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 3.86863431
Iteration 2, loss = 37.94944888
Iteration 3, loss = 58.53828978
Iteration 4, loss = 75.36583180
Iteration 5, loss = 81.58247956
Iteration 6, loss = 75.52996649
Iteration 7, loss = 63.93499237
Iteration 8, loss = 52.49652669
Iteration 9, loss = 43.18005082
Iteration 10, loss = 35.08563195
Iteration 11, loss = 27.53375430
Iteration 12, loss = 21.18930988
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 3.85309285
Iteration 2, loss